# Ag-Analytics-Harmonised Landsat Sentinel API-Demo


<br>The Harmonized Landsat and Sentinel-2 (HLS) is a NASA initiative to produce a Virtual Constellation (VC) of surface reflectance (SR) data from the Operational Land Imager (OLI) and MultiSpectral Instrument (MSI) onboard the Landsat 8 and Sentinel-2 remote sensing satellites, respectively. The data from these satellites creates unprecedented opportunities for timely and accurate observation of Earth status and dynamics at moderate (<30 m) spatial resolution every 2-3 days.

<br>The Ag-Analytics provides the service which a user can provide a polygon in specific formats in additional with more customized options to retrieve the dynamics of their land at various times for Landsat from 2013 and Sentinel-2 from 2015 calendar years. Additionally, this service also provides information on cloud cover, statistics, Normalized Difference Vegetation Index in addition to MSI bands information.

<br>

<br> Experience the usage of this service on Ag-Analytics website using below link
<br>https://ag-analytics.portal.azure-api.net/docs/services/harmonized-landsat-sentinel-service/operations/hls-service

In [1]:
import requests
import json
import time
from pandas.io.json import json_normalize
from collections import defaultdict
import pandas as pd
import zipfile, io
from IPython.display import Image

%matplotlib inline
%autosave 0

Autosave disabled


###### Request parameters details

Request URL: https://aganalyticsapimanagementservice.azure-api.net/hls/detect_and_clip_tiles


1) aoi (geometry, file/text, required): The structure of the geometry can be one of the following.
<br>
i.	  JSON geometry objects returned by the arcgis rest api, (file/text) 
ii.	  GEOJSON (file/text)
iii.  Shapefile (file)
iv.	  Raster of tiff extension (file)


2) Band (Spectral band name, list, required): Provide the list of HLS Spectral band names to retrieve for given aoi<br>
•	Currently Supporting: Red, Green, Blue, NDVI, Coastal Aerosol, NIR


3) satellite (Type of satellite, text, optional):<br>  Landsat or Sentinel. Default (Landsat)

4) resolution (cellsize in meters, float, optional):<br> Default (0.0001)


5) Startdate (Date, mm/dd/yyyy, optional): Enter the starting date to capture the details from Landsat/Sentinel-2<br>
•	Year in Startdate for Landsat should start from 2013 <br>
•	Year in Startdate for Sentinel should start from 2015 <br>
•	Note Startdate and Enddate both should be given, In the absence of other the service retrieves the latest information available on the land.<br>

6) Enddate (Date, mm/dd/yyyy, optional): Enter the end date to capture the details from Landsat/Sentinel-2<br>
•	Year in Enddate for Landsat can be till current year <br>
•	Year in Enddate for Sentinel can be till current year <br>
•	Note Startdate and Enddate both should be given, In the absence of other the service retrieves the latest information available on the land<br>

7) byweek (int,boolean,optional):This parameter will mosaic the rasters in a week Default 1


In order to get Ocp-Apim-Subscription-Key, please click on this link https://ag-analytics.org/developer/Session/SignInFromPayment

### Request Parameters

In [2]:
values={"aoi": '{"type":"Feature","geometry":{"type":"Polygon","coordinates":[[[-93.511545,42.071053],[-93.511565,42.074566],[-93.50667,42.074588],[-93.501908,42.074559],[-93.501936,42.071045],[-93.511545,42.071053]]]},"properties":{"OBJECTID":3350330,"CALCACRES":77.09999847,"CALCACRES2":null},"id":3350330}',
"Band": "['NDVI']",
"Enddate": "3/8/2019",
"Startdate": "3/2/2019",
"legendtype": "Relative",
"satellite": "Landsat,Sentinel",
"resolution":"0.00001",
"byweek":"1"}


    
headers={'Content-Type':'application/x-www-form-urlencoded','Ocp-Apim-Subscription-Key': 'xxxxxxxxxxxxxxxxxxxxxxxxxxxxxx'}
           

###  API Function

In [3]:
def harmonisedlandsatdata(values,headers):
    try:
        url = "https://aganalyticsapimanagementservice.azure-api.net/detect_and_clip_tiles"
        
     
        response = requests.post(url, data=values,headers=headers).json()
        
        print(response)

        return response
    
    except Exception as e:
        print(e)
        raise e

### Calling API Function and Displaying Response

In [4]:
hlsresponse=harmonisedlandsatdata(values,headers)

[{'Values': '', 'Xcoordinates': '', 'Ycoordinates': '', 'band': 'NDVI', 'download_url': 'downloads/raster_bandNDVI_date2019196-2019202_20190807_224634_3097.tif', 'error': '', 'features': [{'attributes': {'CellSize': [1e-05, -1e-05], 'CoordinateSystem': 'GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AUTHORITY["EPSG","4326"]]', 'Extent': '-93.51123513090414, 42.07088071607389, -93.50159513090414, 42.074410716073885', 'Legend': [{'Area': '0.91 %', 'Count': 2577, 'CountAllPixels': 282639, 'Max': 0.2587678168353666, 'Mean': 0.23697687801293443, 'Min': 0.21518593919050227, 'color': '#ff0000'}, {'Area': '1.86 %', 'Count': 5255, 'CountAllPixels': 282639, 'Max': 0.30234969448023097, 'Mean': 0.28055875565779875, 'Min': 0.2587678168353666, 'color': '#ff6666'}, {'Area': '8.33 %', 'Count': 23546, 'CountAllPixels': 28

### Displaying  Output Image

In [5]:


for resp in hlsresponse:
    if resp["band"]=="NDVI" and resp["download_url"]!="":
        
        print("Legend Values : ",resp["features"][0]['attributes']['Legend'])
        
        img_uri=resp["features"][0]['attributes']["pngb64"]
        
        Image(url=img_uri)

        
        

Image(url=img_uri)        
        

Legend Values :  [{'Area': '0.91 %', 'Count': 2577, 'CountAllPixels': 282639, 'Max': 0.2587678168353666, 'Mean': 0.23697687801293443, 'Min': 0.21518593919050227, 'color': '#ff0000'}, {'Area': '1.86 %', 'Count': 5255, 'CountAllPixels': 282639, 'Max': 0.30234969448023097, 'Mean': 0.28055875565779875, 'Min': 0.2587678168353666, 'color': '#ff6666'}, {'Area': '8.33 %', 'Count': 23546, 'CountAllPixels': 282639, 'Max': 0.3459315721250953, 'Mean': 0.32414063330266313, 'Min': 0.30234969448023097, 'color': '#ffff66'}, {'Area': '22.14 %', 'Count': 62577, 'CountAllPixels': 282639, 'Max': 0.3895134497699596, 'Mean': 0.36772251094752745, 'Min': 0.3459315721250953, 'color': '#ffff00'}, {'Area': '48.11 %', 'Count': 135985, 'CountAllPixels': 282639, 'Max': 0.433095327414824, 'Mean': 0.4113043885923918, 'Min': 0.3895134497699596, 'color': '#66ff66'}, {'Area': '18.65 %', 'Count': 52699, 'CountAllPixels': 282639, 'Max': 0.4766772050596883, 'Mean': 0.45488626623725614, 'Min': 0.433095327414824, 'color': '#